In [1]:
%matplotlib inline
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.utils import Sequence
from keras.utils import load_img
from keras.optimizers import RMSprop
from keras.optimizers import Adam


2023-01-26 03:13:10.659786: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-26 03:13:12.452912: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /appl/cudnn/v8.3.2.44-prod-cuda-11.5/lib:/appl/cuda/11.6.0/lib64:/appl/python/3.10.7/lib:/appl/gcc/11.3.0-binutils-2.38/lib64:/appl/gcc/11.3.0-binutils-2.38/lib:/lsf/10.1/linux3.10-glibc2.17-x86_64/lib
2023-01-26 03:13:12.453554: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so

In [2]:
!nvidia-smi

Thu Jan 26 03:13:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:37:00.0 Off |                    0 |
| N/A   35C    P0    38W / 250W |   7536MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:AF:00.0 Off |                    0 |
| N/A   

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
df = pd.read_csv('black_background_500x500.csv')
train_df = df[df['ImagePath'].str.contains("train")]
test_df = df[df['ImagePath'].str.contains("test")]
valid_df = df[df['ImagePath'].str.contains("valid")]

input_size = 500

In [5]:
#########################################################
################## data generator #######################
#########################################################
class datagenerator(tf.keras.utils.Sequence):
    def __init__(self, 
            batch_size, 
            img_size,
            data_paths_df,
            input_channels,
            output_channels):
         
        self.batch_size = batch_size
        self.img_size = img_size
        self.data_paths_df = data_paths_df
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.data_paths = data_paths_df.values[:,1]
        self.params = data_paths_df.values[:,3:6]
        assert len(self.data_paths) == len(self.params)
        
        self.n = len(self.data_paths)

    def on_epoch_end(self):
        'updates indexes after each epoch'
        self.data_paths_df = self.data_paths_df.sample(frac = 1)
        self.data_paths = self.data_paths_df.values[:,1]
        self.params = self.data_paths_df.values[:,3:6]
    
    def __getitem__(self, index):
        batch_data_paths = self.data_paths[index : index + self.batch_size]
        batch_params_paths = self.params[index : index + self.batch_size]

        return self.__dataloader(self.img_size,
                batch_data_paths, batch_params_paths,
                self.input_channels, self.output_channels)
    
    def __len__(self):
        return self.n // self.batch_size

    #################### data loader ########################
    def __dataloader(self, 
            img_size,
            data_paths,
            batch_params_paths,
            input_channels,
            output_channels):
        x = np.zeros((len(data_paths), img_size[0], img_size[1], input_channels))
        y = batch_params_paths        
        
        for i in range(len(data_paths)):
            data = load_img(path = data_paths[i], color_mode = "grayscale")
            data = tf.keras.utils.img_to_array(data, data_format="channels_last", dtype="float32")
            data /= 255
#             data.shape = (1,) + data.shape
            x[i] = np.asarray(data)
        return x.astype("float32"), np.array(y).astype("float32")

In [6]:
inputs = keras.Input(shape=(input_size, input_size, 1))
x = layers.Conv2D(filters=16, kernel_size=11, activation="relu")(inputs)
x = layers.Conv2D(filters=16, kernel_size=11, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=4)(x)
x = layers.Conv2D(filters=16, kernel_size=5, activation="relu")(x)
x = layers.Conv2D(filters=16, kernel_size=5, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=4)(x)
x = layers.Conv2D(filters=16, kernel_size=5, activation="relu")(x)
x = layers.Conv2D(filters=16, kernel_size=5, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(3)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500, 500, 1)]     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 490, 490, 16)      1952      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 122, 122, 16)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 118, 118, 16)      6416      
                                                                 
 conv2d_3 (Conv2D)           (None, 114, 114, 16)      6416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 16)       0         
 2D)                                                         

2023-01-26 03:13:16.775778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-26 03:13:17.499952: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14610 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:af:00.0, compute capability: 7.0


In [39]:
inputs = keras.Input(shape=(input_size, input_size, 1))
x = layers.Conv2D(filters=4, kernel_size=3, activation="relu")(inputs)
x = layers.Conv2D(filters=4, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=8)(x)
x = layers.Conv2D(filters=4, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=4, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=4)(x)
x = layers.Conv2D(filters=4, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=4, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation="relu")(x) 
outputs = layers.Dense(3)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 500, 500, 1)]     0         
                                                                 
 conv2d_98 (Conv2D)          (None, 498, 498, 4)       40        
                                                                 
 conv2d_99 (Conv2D)          (None, 496, 496, 4)       148       
                                                                 
 max_pooling2d_48 (MaxPoolin  (None, 62, 62, 4)        0         
 g2D)                                                            
                                                                 
 conv2d_100 (Conv2D)         (None, 60, 60, 4)         148       
                                                                 
 conv2d_101 (Conv2D)         (None, 58, 58, 4)         148       
                                                          

In [40]:
import keras.backend as K
  
def abs_loss_function(y_true, y_pred):   
    abs_diff = K.abs(y_true - y_pred)
    ones = tf.ones_like(y_true)
    abs_diff_reversed = K.abs(ones - abs_diff )   
    minimum_from_two = tf.math.minimum(abs_diff, abs_diff_reversed) 
    return tf.math.reduce_mean(minimum_from_two, axis=-1)

def square_abs_min_loss(y_true, y_pred):   
    abs_diff = K.abs(y_true - y_pred)
    ones = tf.ones_like(y_true)
    abs_diff_reversed = K.abs(tf.ones_like(y_true) - abs_diff )   
    minimum_from_two = tf.math.minimum(abs_diff, abs_diff_reversed) 
    min_sq = tf.math.sqrt(minimum_from_two)
    return tf.math.reduce_mean(min_sq, axis=-1) 

def smart_square_abs_min_loss(y_true, y_pred):  
    punished_y_pred = tf.where((y_pred<0)|(y_pred>1), 3.0 + K.abs(y_pred),y_pred)
    
    abs_diff = K.abs(y_true - punished_y_pred)
    ones = tf.ones_like(y_true)
    abs_diff_reversed = K.abs(ones - abs_diff)   
    minimum_from_two = tf.math.minimum(abs_diff, abs_diff_reversed) 
#     print("_________________ 1 __________________")
#     print(abs_diff.numpy())
#     print("_________________ 2 __________________")
#     print(y_pred.numpy())
#     print("_________________ 3 __________________")
#     print(punished_y_pred.numpy())
    
    return tf.math.reduce_mean(minimum_from_two, axis=-1)
    
############################# For debugging ####################################
#     print("_________________ 1 __________________")
#     print(abs_diff_reversed.numpy())
#     print("_________________ 2 __________________")
#     print(abs_diff.numpy())

In [41]:
from keras.callbacks import LambdaCallback
test_g = datagenerator(32, (input_size,input_size), test_df, 1, 3)
def evaluate_model(self, epoch, logs=None):
    model.evaluate(test_g, batch_size=32)
    print(self)
    
evaluate_callback = LambdaCallback(on_epoch_end=evaluate_model, period=1)

In [16]:
from keras.callbacks import *

In [42]:

class LossHistory(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 10 == 0:
            print("-------------------------------------------------------")
            model_e = self.model.evaluate(test_g, batch_size=32)
            print(model_e[1])
            print("-------------------------------------------------------")

In [43]:
model.compile(optimizer = RMSprop(learning_rate=0.001),
              loss = abs_loss_function, 
              metrics = [abs_loss_function, "accuracy"],
              run_eagerly=True)  # Add run_eagerly=True to enable the numpy debugging

tg = datagenerator(32, (input_size,input_size), train_df, 1, 3)
vg = datagenerator(32, (input_size,input_size), valid_df, 1, 3)
history = model.fit(x=tg,
                    batch_size=32,
                    epochs=1,
                    validation_data=vg,
                    callbacks=[LossHistory()])

31/31 [==============================] - 6s 199ms/step - loss: 0.2545 - abs_loss_function: 0.2545 - accuracy: 0.3962
[0.2545084059238434, 0.2545084059238434, 0.39616936445236206]
-------------------------------------------------------
273/273 [==============================] - 89s 323ms/step - loss: 0.2359 - abs_loss_function: 0.2359 - accuracy: 0.3043 - val_loss: 0.2660 - val_abs_loss_function: 0.2660 - val_accuracy: 0.3024


In [44]:
model_e = model.evaluate(test_g, batch_size=32)

31/31 [==============================] - 8s 243ms/step - loss: 0.2404 - abs_loss_function: 0.2404 - accuracy: 0.2782


In [45]:
model.evaluate(test_g, batch_size=32)


31/31 [==============================] - 8s 240ms/step - loss: 0.2680 - abs_loss_function: 0.2680 - accuracy: 0.4808


[0.26796549558639526, 0.26796549558639526, 0.4808467626571655]

In [46]:
model_e[1]

0.2404078245162964

In [ ]:
model.compile(optimizer = RMSprop(learning_rate=0.001),
              loss = square_abs_min_loss, 
              metrics = [square_abs_min_loss, "accuracy"],
              run_eagerly=True)  # Add run_eagerly=True to enable the numpy debugging

tg = datagenerator(32, (input_size,input_size), train_df, 1, 3)
vg = datagenerator(32, (input_size,input_size), valid_df, 1, 3)
history2 = model.fit(x=tg,
                    batch_size=32,
                    epochs=50,
                    validation_data=vg)

In [ ]:
model.compile(optimizer = RMSprop(learning_rate=0.001),
              loss = square_abs_min_loss, 
              metrics = [square_abs_min_loss, "accuracy"],
              run_eagerly=True)  # Add run_eagerly=True to enable the numpy debugging

tg = datagenerator(32, (input_size,input_size), train_df, 1, 3)
vg = datagenerator(32, (input_size,input_size), valid_df, 1, 3)
history3 = model.fit(x=tg,
                    batch_size=32,
                    epochs=50,
                    validation_data=vg)

In [ ]:
def plot_accuracy_loss(history, name):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(12,5))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['accuracy'],'bo--', label = "acc")
    plt.plot(history.history['val_accuracy'],'ro--', label = "val_accuracy")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()
    
    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'],'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
plot_accuracy_loss(history, "1")

In [ ]:
def load_model_a():
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=4)(x)
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=4)(x)
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=4)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dense(16, activation="relu")(x)
    outputs = layers.Dense(3)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


def load_model_b():
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = layers.Conv2D(filters=16, kernel_size=11, activation="relu")(inputs)
    x = layers.Conv2D(filters=16, kernel_size=7, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=16, kernel_size=5, activation="relu")(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dense(16, activation="relu")(x)
    outputs = layers.Dense(3)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

def load_model_c():
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = layers.Conv2D(filters=16, kernel_size=11, activation="relu")(inputs)
    x = layers.Conv2D(filters=16, kernel_size=7, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=16, kernel_size=5, activation="relu")(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01))(x)
    x = layers.Dense(16, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01))(x) 
    outputs = layers.Dense(3)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


def plot_accuracy_loss(history):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(12,5))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['accuracy'],'bo--', label = "acc")
    plt.plot(history.history['val_accuracy'],'ro--', label = "val_accuracy")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()
    
    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'],'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()

def load_and_train_model(model, epochs = 100, loss_func = abs_loss_function):
    tg = datagenerator(32, (input_size,input_size), train_df, 1, 3)
    vg = datagenerator(32, (input_size,input_size), valid_df, 1, 3)
    model.compile(optimizer = RMSprop(learning_rate=0.001),
                  loss = loss_func, 
                  metrics = ["accuracy", loss_func])  # Add run_eagerly=True to enable the numpy debugging
    
    history = model.fit(x=tg,
                        batch_size=32,
                        epochs=epochs,
                        validation_data=vg)
    
    
    plot_accuracy_loss(history)
    return history

In [ ]:
func_m = eval("load_model_c")
model_ask = func_m()
model_ask.summary()

In [ ]:
model_a = load_model_a()
model_a.summary()

In [ ]:
history_a = load_and_train_model(model_a, 100, abs_loss_function)

In [ ]:
model_b = load_model_b()
model_b.summary()

In [ ]:
history_b = load_and_train_model(model_b, 100, abs_loss_function)

In [ ]:
model_c = load_model_c()
model_c.summary()

In [ ]:
history_c = load_and_train_model(model_c, 100, abs_loss_function)

In [ ]:
def plot_accuracy_loss(history, name):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(12,5))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['accuracy'],'bo--', label = "acc")
    plt.plot(history.history['val_accuracy'],'ro--', label = "val_accuracy")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()
    
    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'],'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()

def load_and_train_model(model, epochs = 100, loss_func = abs_loss_function):
    tg = datagenerator(32, (input_size,input_size), train_df, 1, 3)
    vg = datagenerator(32, (input_size,input_size), valid_df, 1, 3)
    model.compile(optimizer = RMSprop(learning_rate=0.001),
                  loss = loss_func, 
                  metrics = ["accuracy", loss_func])  # Add run_eagerly=True to enable the numpy debugging
    
    history = model.fit(x=tg,
                        batch_size=32,
                        epochs=epochs,
                        validation_data=vg)
    
    
    plot_accuracy_loss(history)
    plt.savefig("{}.png".format(name))
    return history

In [ ]:
plot_accuracy_loss(history_a)
plot_accuracy_loss(history_b)
plot_accuracy_loss(history_c)

In [ ]:
model.summary()

In [ ]:
tg = datagenerator(32, (input_size,input_size), test_df, 1, 3)
results = model.evaluate(tg, batch_size=32)
print("test loss, test acc:", results)

In [ ]:
predictions = model.predict(tg)  
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability

In [ ]:
predictions

In [ ]:
t = test_df.values[1][1]
data = load_img(path = t, grayscale = True)
data = tf.keras.utils.img_to_array(data, data_format="channels_last", dtype="float32")
data /= 255
data.shape = (1,) + data.shape
X = np.asarray(data)

print(t)




def predict_data(d):
    yhat = model.predict(data)
    yhat2 = np.where(yhat<0, 1+yhat, yhat)
    print(yhat2*90)

predict_data(data)

In [ ]:
def prdict_and_print(nr):
    t = test_df.values[nr][1]
    data = load_img(path = t, grayscale = True)
    data = tf.keras.utils.img_to_array(data, data_format="channels_last", dtype="float32")
    data /= 255
    data.shape = (1,) + data.shape
    X = np.asarray(data)
    print("----------{}----------".format(nr))
    euler = test_df.values[nr]
    print("phi1", float(euler[3])*90)
    print("PHI",   float(euler[4])*90)
    print("phi2",  float(euler[5])*90)
    yhat = model.predict(data)
    print("predicted values", yhat*90)

    
print("############### PREDICTIONS ###############")
for i in range(10):
    prdict_and_print(i)
print("############### PREDICTIONS ###############")

In [ ]:
model_name = "Best_model_so_far_abs_loss_function_RMSprop"
model.save("Models/{}.h5".format(model_name), save_format = 'h5')

In [ ]:
def plot_accuracy_loss(history):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(12,5))

    # Plot accuracy
#     plt.subplot(221)
#     plt.plot(history.history['accuracy'],'bo--', label = "acc")
#     plt.plot(history.history['val_accuracy'],'ro--', label = "val_accuracy")
#     plt.title("train_acc vs val_acc")
#     plt.ylabel("accuracy")
#     plt.xlabel("epochs")
#     plt.legend()
    
    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'],'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
#     plt.savefig("graph_{}_{}_{}_{}.png".format(input_size,input_size,input_size,input_size))
plot_accuracy_loss(history)